In [ ]:
## This file is meant to be run only once
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import descartes
from shapely.geometry import Polygon
import geopandas as gpd
import os.path
from PIL import Image
import rasterio
import math

# read in all of your files #TODO make sure these match your file path
os.chdir("/Users/aminaly/Box Sync/mountain_biodiversity")

In [ ]:
#### read in and combine all of the wdpa files. ONLY NEEDS TO RUN ONCE

## read in all the wdpa files
wdpa0 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_0/WDPA_May2021_Public_shp-polygons.shp")
wdpa1 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_1/WDPA_May2021_Public_shp-polygons.shp")
wdpa2 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_2/WDPA_May2021_Public_shp-polygons.shp")
wdpa3 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_0/WDPA_May2021_Public_shp-points.shp")
wdpa4 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_1/WDPA_May2021_Public_shp-points.shp")
wdpa5 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public_shp_2/WDPA_May2021_Public_shp-points.shp")

#combine the poly and point dataframes
wdpa_poly = gpd.GeoDataFrame(pd.concat([wdpa0, wdpa1, wdpa2]))
wdpa_point = gpd.GeoDataFrame(pd.concat([wdpa3, wdpa4, wdpa5]))

#save these out so we don't have to re-run again
wdpa_poly.to_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-polygons.shp", driver='ESRI Shapefile')
wdpa_point.to_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-points.shp", driver='ESRI Shapefile')

wdpa_poly.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}

In [ ]:
#### If you don't need to run the above cell, here are the final poly and points to work qith
wdpa_poly = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-polygons.shp", driver='ESRI Shapefile')
wdpa_point = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_May2021_Public_shp/WDPA_May2021_Public/WDPA_May2021_Public_shp-points.shp", driver='ESRI Shapefile')


In [ ]:
#### Take in points and create buffers around them based on reported area

## Add buffers to the point based on their area
wdpa_point.crs = {'init': 'epsg:3763', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:3763'})

## calculate the radius
wdpa_point['radius'] = np.sqrt(wdpa_point.REP_AREA / np.pi )

## remove entries with 0 reported area/radius
wdpa_point.drop(wdpa_point[wdpa_point['radius'] <= 0].index, inplace = True)

## create the buffers
wdpa_point['geometry'] = wdpa_point.geometry.buffer(wdpa_point.radius)

# convert projection back
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:4326'})